In [4]:
from dataclasses import dataclass
from typing import List, Callable, Dict


In [83]:

@dataclass
class Token:
    '''Class representing a token with <type> and <value>

    Attributes:
        type (str): The token type.
        value (str): The token value.
    '''
    type: str
    value: str
#     target: str = ''

    def __repr__(self):
        return '({0}: {1})'.format(self.type, self.value)


@dataclass
class Block:
    '''Class representing a block with an <id> and <content>

    Attributes:
        id (str): A unique block id, likely a hash of its content. 
        content (List[Token]): The block's content; a list of tokens.
        indent (int): **this should probably be removed**
    '''
    id: str
    content: List[Token]
#     indent: int

    def __eq__(self, other) -> bool:
        return self.id == other.id

    def __repr__(self):
        return self.id + ': ' + ' '.join([x.value for x in self.content])


In [84]:
print(Token('REF', 'George Washington'))
print(Block('a', [Token('REF', 'George Washington')]))

(REF: George Washington)
a: George Washington


In [91]:
import re

def find_groups(text, open_ref = r'\[\[', close_ref = r'\]\]'):
    groups = []
    x = 0
    y = 0
    while re.search(open_ref, text[y:]) and re.search(close_ref, text[y:]):
        x = re.search(open_ref, text[y:]).span()[0] + y
        groups.append(Token('txt', text[y:x]))
        y = re.search(close_ref, text[x:]).span()[1] + x
        groups.append(Token('ref', text[x:y]))
    groups.append(Token('txt', text[y:]))
    return groups

print(find_groups('hello [[world]], how are you'))
print(find_groups('goodbye [[moon]], goodbye [[stars]]'))
print(find_groups('cya \wave !', open_ref=r'\\', close_ref=r'\s|$'))

[(txt: hello ), (ref: [[world]]), (txt: , how are you)]
[(txt: goodbye ), (ref: [[moon]]), (txt: , goodbye ), (ref: [[stars]]), (txt: )]
[(txt: cya ), (ref: \wave ), (txt: !)]


In [103]:
groups = {
    'ref': {
        'open': r'\[\[',
        'close': r'\]\]'
    },
    'keyword': {
        'open': r'\\',
        'close': r'\s|$'
    }
}


text = 'goodbye [[moon]], goodbye [[stars]]'
tokens = []
while True:
    index = 0
    matchtype = None
    y = 0
    for gname in groups.keys():
        match = re.search(groups[gname]['open'], text[index:])
        if match:
            x = match.span()[0]
            if x < index:
                index = x
                matchtype = 
    print(text[index:])

goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[

goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[

goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[

goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[

goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[

goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[

goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[stars]]
goodbye [[moon]], goodbye [[

KeyboardInterrupt: 